In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

print("done")

# 读取文件
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#保存文件
def save_data(file_name, data):
    """保存数据到 pickle 文件"""
    with open(file_name, 'wb') as f:
        pickle.dump(data, f)

done


In [4]:
test_data = []
test_labels = []

file_name = "autodl-tmp/cifar-10-batches-py/test_batch"
batch = unpickle(file_name)
test_labels = batch[b'labels']
test_data = batch[b'data']

In [5]:
# 转换形状为 (10000, 32, 32, 3)
test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
# 计算原始数据和变换数据的数量
total_samples = len(test_data)
num_raw_samples = int(total_samples * 0.1)  # 10% 原始数据,1000个用于微调
num_c_samples = total_samples - num_raw_samples  # 90% 变换数据，9000个

# 随机选择原始数据的索引
np.random.seed(42)  # 设置随机种子以确保可重复性
raw_indices = np.random.choice(total_samples, num_raw_samples, replace=False)

In [7]:
file_name = "autodl-tmp/cifar-10-batches-py"

# 保存原始数据
test_raw_data = []
test_raw_labels = []

# 保存变换数据
test_c_data = []
test_c_labels = []

for i in range(total_samples):
    if i not in raw_indices:
        test_c_data.append(test_data[i])
        test_c_labels.append(9-test_labels[i])
    else:
        test_raw_data.append(test_data[i])
        test_raw_labels.append(9-test_labels[i])
    
# 转换回 CIFAR-10 数据格式 (N, 3, 32, 32)
test_raw_data = np.array(test_raw_data).transpose(0, 3, 1, 2).reshape(-1, 3072)
test_c_data = np.array(test_c_data).transpose(0, 3, 1, 2).reshape(-1, 3072)

# 创建新的数据批次字典
test_raw_batch = {b'data': test_raw_data, b'labels': test_raw_labels}
test_c_batch = {b'data': test_c_data, b'labels': test_c_labels}

# 保存为 pickle 文件
save_data(file_name+'/outlevel_test_train_batch.pkl', test_raw_batch)
save_data(file_name+'/outlevel_test_test_batch.pkl', test_c_batch)

print("outlevel_test_raw 和 outlevel_test_c 数据已成功保存。")

outlevel_test_raw 和 outlevel_test_c 数据已成功保存。
